Comp-551 A3 
</br>
We divide the prediction of handwriten digits and letters into two parts. Here is the letter prediction part.

In [ ]:
! gdown https://drive.google.com/uc?id=1AbxjCBByF-xGV3c7FJ1ztZC1HO0O0uJ0

Downloading...
From: https://drive.google.com/uc?id=1AbxjCBByF-xGV3c7FJ1ztZC1HO0O0uJ0
To: /content/kaggle.json
100% 66.0/66.0 [00:00<00:00, 61.3kB/s]


In [ ]:
! pip install kaggle
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'
! kaggle competitions download -c comp-551-fall-2021
! unzip images_l.pkl.zip
! unzip images_test.pkl.zip
! unzip images_ul.pkl.zip
! unzip labels_l.pkl.zip

images_test.pkl.zip: Skipping, found more recently modified local copy (use --force to force download)
images_l.pkl.zip: Skipping, found more recently modified local copy (use --force to force download)
images_ul.pkl.zip: Skipping, found more recently modified local copy (use --force to force download)
labels_l.pkl.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  images_l.pkl.zip
replace images_l.pkl? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
Archive:  images_test.pkl.zip
replace images_test.pkl? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
Archive:  images_ul.pkl.zip
replace images_ul.pkl? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
Archive:  labels_l.pkl.zip
replace labels_l.pkl? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
import io
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, LeakyReLU, ZeroPadding2D,Convolution2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import History
from tensorflow.keras.optimizers import Nadam, Adam, SGD
from keras.applications.vgg16 import VGG16
from keras.applications.densenet import DenseNet169
from keras.applications.resnet import ResNet152, ResNet101, ResNet50
import matplotlib.pyplot as plt
from keras.utils import np_utils
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder

import tensorflow as tf
from google.colab import drive
import numpy as np
import h5py
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras import layers, regularizers, Input
from keras.layers import *
from keras.callbacks import ReduceLROnPlateau
from keras import optimizers

In [ ]:
# Image data for training (30,000 sample, each sample is a 56x56 image)
# train_labels = pd.read_csv('train_max_y.csv')
# train_images = pd.read_pickle('train_max_x')
# test_images = pd.read_pickle('test_max_x')
with open("images_l.pkl", 'rb') as f:
    # load into as a numpy array
    train_labelled_data = pickle.load(f)
# Test images. The prediction corresponding to these images should be uploaded. (15,000 samples)
with open("images_test.pkl", 'rb') as f:
    test_data = pickle.load(f)
# Labels for training (30,000 rows, each row is a size 36 binary vector, which is the label to the corresponding image)
with open("labels_l.pkl", 'rb') as f:
    train_labelled_y = pickle.load(f)
# Additional images that can be used for training the classifier.
# Labels for these images are not provided. (30,000 samples, where each sample is a 56x56 image)
with open("images_ul.pkl", 'rb') as f:
    train_unlabelled_x = pickle.load(f)

# Processing

Model is not noise-sensitive, so we do not use the add_noise data or clear_noise data

# Letter

In [ ]:
def cnn2():
  model = Sequential()
  model.add(Input(shape=(56, 56, 1)))
  model.add(Conv2D(filters= 56, kernel_size=(3,3), padding='Same', activation='relu'))
  model.add(Conv2D(filters= 56, kernel_size=(3,3), padding='Same', activation='relu'))
  model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
  model.add(Dropout(0.6))

  model.add(Conv2D(filters= 128, kernel_size=(3,3), padding='Same', activation='relu'))
  model.add(Conv2D(filters= 128, kernel_size=(3,3), padding='Same', activation='relu'))
  model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
  model.add(Dropout(0.6))

  model.add(Conv2D(filters= 256, kernel_size=(3,3), padding='Same', activation='relu'))
  model.add(Conv2D(filters= 256, kernel_size=(3,3), padding='Same', activation='relu'))
  model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
  model.add(Dropout(0.6))

  model.add(Conv2D(filters= 512, kernel_size=(3,3), padding='Same', activation='relu'))
  model.add(Conv2D(filters= 512, kernel_size=(3,3), padding='Same', activation='relu'))
  model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
  model.add(Dropout(0.6))

  model.add(Flatten())
  model.add(Dense(1024, activation='relu'))
  model.add(Dropout(0.6))
  model.add(Dense(26, activation='softmax'))
  model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
  return model
model2=cnn2()
model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 56, 56, 56)        560       
                                                                 
 conv2d_17 (Conv2D)          (None, 56, 56, 56)        28280     
                                                                 
 batch_normalization_10 (Bat  (None, 56, 56, 56)       224       
 chNormalization)                                                
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 28, 28, 56)       0         
 2D)                                                             
                                                                 
 dropout_10 (Dropout)        (None, 28, 28, 56)        0         
                                                                 
 conv2d_18 (Conv2D)          (None, 28, 28, 128)      

In [ ]:
# # reload train data for model2 training
# train_labelled_x=np.array(pd.DataFrame(pd.read_csv('processed_train_set.csv')))

# train_labelled_x=train_labelled_x.reshape((30000,56,56))

# import matplotlib.pyplot as plt
# import random

# plt.imshow(train_labelled_x[random.randint(0,29999)], cmap="gray")
# plt.show()

with open("images_l.pkl", 'rb') as f:
    # load into as a numpy array
    train_labelled_data = pickle.load(f)

# Labels for training (30,000 rows, each row is a size 36 binary vector, which is the label to the corresponding image)
with open("labels_l.pkl", 'rb') as f:
    train_labelled_y = pickle.load(f)

train_labelled_x = train_labelled_data

In [ ]:
letter_train = train_labelled_x.reshape(-1, 56, 56, 1)
# letter_train /= 255
letter_train_label = train_labelled_y[:, 10:]
x_train2 = letter_train[:29000,:,:,:]
y_train2 = letter_train_label[:29000,:]
x_valid2 = letter_train[29000:,:,:,:]
y_valid2 = letter_train_label[29000:,:]

reduce_lr = ReduceLROnPlateau(monitor='val_dense_1_accuracy', patience=2, verbose=1,factor=0.8, min_lr=0.000001)

In [ ]:
history = model2.fit(x_train2, y_train2, epochs=25, batch_size=128,
          validation_data=(x_valid2, y_valid2), 
          callbacks=[reduce_lr], shuffle=False)

Epoch 1/25
227/227 [==============================] - 15s 58ms/step - loss: 4.0627 - accuracy: 0.0731 - val_loss: 6.3259 - val_accuracy: 0.0470 - lr: 0.0010
Epoch 2/25
227/227 [==============================] - 13s 57ms/step - loss: 3.3783 - accuracy: 0.1083 - val_loss: 6.2230 - val_accuracy: 0.0440 - lr: 0.0010
Epoch 3/25
227/227 [==============================] - 13s 57ms/step - loss: 3.0401 - accuracy: 0.1369 - val_loss: 2.8367 - val_accuracy: 0.1680 - lr: 0.0010
Epoch 4/25
227/227 [==============================] - 13s 57ms/step - loss: 2.7621 - accuracy: 0.1854 - val_loss: 4.6307 - val_accuracy: 0.0960 - lr: 0.0010
Epoch 5/25
227/227 [==============================] - 13s 57ms/step - loss: 2.4098 - accuracy: 0.2738 - val_loss: 3.1373 - val_accuracy: 0.2520 - lr: 0.0010
Epoch 6/25
227/227 [==============================] - 13s 57ms/step - loss: 1.9758 - accuracy: 0.3842 - val_loss: 2.1136 - val_accuracy: 0.3800 - lr: 0.0010
Epoch 7/25
227/227 [==============================] - 13s 

In [ ]:
history = model2.fit(x_train2, y_train2, epochs=1, batch_size=128,
          validation_data=(x_valid2, y_valid2), 
          callbacks=[reduce_lr], shuffle=False)

227/227 [==============================] - 13s 58ms/step - loss: 0.0998 - accuracy: 0.9680 - val_loss: 0.1250 - val_accuracy: 0.9680 - lr: 0.0010


In [ ]:
pickle.dump(model2, open('model2_1.pkl', 'wb'))

INFO:tensorflow:Assets written to: ram://800fb314-90b9-46de-96b5-660b8b0696d3/assets


In [ ]:
# # load testing data for model testing
# x_test_data=np.array(pd.DataFrame(pd.read_csv('processed_test_set.csv')))

# x_test_data=x_test_data.reshape((15000,56,56))

# import matplotlib.pyplot as plt
# import random

# plt.imshow(x_test_data[random.randint(0,14999)], cmap="gray")
# plt.show()

with open("images_test.pkl",'rb') as f:
  x_test_data = pickle.load(f)
x_test = x_test_data.reshape(-1,56,56,1)
# x_test /= 255

In [ ]:
result2 = model2.predict(x_test, verbose=1)
result2 = np.argmax(result2, axis = 1)